#### requests/Selenium 처리 상의 차이점 소개

## 최신 버전명 찾기

* 크롤링 대상
"https://chromedriver.storage.googleapis.com/index.html"

여기에는 크롬드라이버 버전별로 들어가 있다. 이 중에 가장 최신 버전명을 알아내고자 한다.

## 방법 1: requests

In [2]:
import requests
from bs4 import BeautifulSoup

### 시도 1

In [3]:
url = "https://chromedriver.storage.googleapis.com/index.html"
res = requests.get(url)
html = res.text
soup = BeautifulSoup(html, "html.parser")
soup.select('a')

[]

### JavaScript에 의한 컨텐츠 처리
이 사이트는 자바스크립코드로 이루어져 있어, 소스코드를 보면 데이터가 보이지 않는다. 이 경우 개발자 도구의 network 상에서 header를 검토한다.
그 결과, GET 방식을 사용하고 있는 XML 형식임을 알 수 있다.

즉, 별도 주소에서 XML 포맷 데이터를 로딩하여 JavaScript를 통해 처리하고 있다.


### 시도 2 : XML 주소로 직접 요청을 보내어 처리

In [4]:
import re

another_url = "https://chromedriver.storage.googleapis.com/?delimiter=/&prefix="
res = requests.get(another_url)
xml = res.text
soup = BeautifulSoup(xml, "html.parser")

In [6]:
res

<Response [200]>

In [15]:
xml[6000:]

's><Prefix>2.24/</Prefix></CommonPrefixes><CommonPrefixes><Prefix>2.25/</Prefix></CommonPrefixes><CommonPrefixes><Prefix>2.26/</Prefix></CommonPrefixes><CommonPrefixes><Prefix>2.27/</Prefix></CommonPrefixes><CommonPrefixes><Prefix>2.28/</Prefix></CommonPrefixes><CommonPrefixes><Prefix>2.29/</Prefix></CommonPrefixes><CommonPrefixes><Prefix>2.3/</Prefix></CommonPrefixes><CommonPrefixes><Prefix>2.30/</Prefix></CommonPrefixes><CommonPrefixes><Prefix>2.31/</Prefix></CommonPrefixes><CommonPrefixes><Prefix>2.32/</Prefix></CommonPrefixes><CommonPrefixes><Prefix>2.33/</Prefix></CommonPrefixes><CommonPrefixes><Prefix>2.34/</Prefix></CommonPrefixes><CommonPrefixes><Prefix>2.35/</Prefix></CommonPrefixes><CommonPrefixes><Prefix>2.36/</Prefix></CommonPrefixes><CommonPrefixes><Prefix>2.37/</Prefix></CommonPrefixes><CommonPrefixes><Prefix>2.38/</Prefix></CommonPrefixes><CommonPrefixes><Prefix>2.39/</Prefix></CommonPrefixes><CommonPrefixes><Prefix>2.4/</Prefix></CommonPrefixes><CommonPrefixes><Prefix>2

In [19]:
version_list = []
for tag in soup.select('prefix'):
    if re.match(r'^\d', tag.text): # 숫자로 시작하는 것인지 확인
        version = tag.text.rstrip('/')
        version_list.append(version)
        
version_list.sort(key=lambda version: tuple(map(int, version.split('.'))))  # 오름 차순 정렬

In [21]:
recent_version = version_list[-1]

In [22]:
print("recent_verson:", recent_version)

recent_verson: 78.0.3904.11


## Selenium 시도 
웹브라우저 자동화를 수행하는 라이브러리로 웹페이지의 현재 DOM 내역을 읽어서 처리한다.

In [23]:
import re
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

In [27]:
chrome_driver_path = "drivers/chromedriver"  

with webdriver.Chrome(executable_path=chrome_driver_path) as driver:  
    # selenium은 직접 요청을 수행하지 않기 때문에 웹드라이버를 크롬을 활용하여 requests를 요청
    url = "https://chromedriver.storage.googleapis.com/index.html"  # 해당 웹페이지 주소를 입력 (xml은 신경 안씀)
    driver.get(url)
    element_list = WebDriverWait(driver, 10).until(                 # 기다리는 시간 10, until은 함수 전달
        EC.presence_of_all_elements_located((By.TAG_NAME, 'a')))  # 주기적으로 체크할 함수 지정, a 태그가 나올때까지 최대 10초까지 기다리겠다는 의미
    version_list = [element.text for element in element_list if re.match(r'^\d', element.text)]

In [28]:
version_list.sort(key=lambda version: tuple(map(int, version.split('.'))))  # 위와 동일

In [29]:
version_list

['2.0',
 '2.1',
 '2.2',
 '2.3',
 '2.4',
 '2.5',
 '2.6',
 '2.7',
 '2.8',
 '2.9',
 '2.10',
 '2.11',
 '2.12',
 '2.13',
 '2.14',
 '2.15',
 '2.16',
 '2.17',
 '2.18',
 '2.19',
 '2.20',
 '2.21',
 '2.22',
 '2.23',
 '2.24',
 '2.25',
 '2.26',
 '2.27',
 '2.28',
 '2.29',
 '2.30',
 '2.31',
 '2.32',
 '2.33',
 '2.34',
 '2.35',
 '2.36',
 '2.37',
 '2.38',
 '2.39',
 '2.40',
 '2.41',
 '2.42',
 '2.43',
 '2.44',
 '2.45',
 '2.46',
 '70.0.3538.16',
 '70.0.3538.67',
 '70.0.3538.97',
 '71.0.3578.30',
 '71.0.3578.33',
 '71.0.3578.80',
 '71.0.3578.137',
 '72.0.3626.7',
 '72.0.3626.69',
 '73.0.3683.20',
 '73.0.3683.68',
 '74.0.3729.6',
 '75.0.3770.8',
 '75.0.3770.90',
 '75.0.3770.140',
 '76.0.3809.12',
 '76.0.3809.25',
 '76.0.3809.68',
 '76.0.3809.126',
 '77.0.3865.10',
 '77.0.3865.40',
 '78.0.3904.11']